In [8]:
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
from torchsummary import summary
import torch
import torch.nn.functional as F
import numpy as np
import importlib 
import config
importlib.reload(config)

<module 'config' from 'e:\\TSAI\\EMLO\\EMLO-session1\\config.py'>

In [9]:
class DatasetFromArray(Dataset):
    def __init__(self, featurepath, labelpath):
        self.data = np.load(open(featurepath, 'rb'))
        self.labels = np.load(open(labelpath, 'rb'))
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        return {'data': torch.tensor(self.data[index], dtype=torch.float32), 'target': torch.tensor(self.labels[index],dtype=torch.long)}
    

In [10]:
train_ds = DatasetFromArray(config.TRAIN_FEATURES, config.TRAIN_LABELS)
valid_ds = DatasetFromArray(config.VALID_FEATURES, config.VALID_LABELS)

In [11]:
train_loader = DataLoader(train_ds, batch_size=16, shuffle=True, pin_memory=True)
valid_loader = DataLoader(valid_ds, batch_size=16, pin_memory=True)

In [12]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Conv2d(512, 64, kernel_size=3, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.Dropout(0.5),
            
            nn.Conv2d(64,2, kernel_size=3, bias=False),
            nn.AvgPool2d(3)

        )
    
    def forward(self, x):
        x = self.model(x)
        x = x.view(-1, 2)
        return F.log_softmax(x, dim=1)

In [13]:

model = Model().to('cuda')
summary(model, input_size=(512,7,7))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1             [-1, 64, 5, 5]         294,912
              ReLU-2             [-1, 64, 5, 5]               0
       BatchNorm2d-3             [-1, 64, 5, 5]             128
           Dropout-4             [-1, 64, 5, 5]               0
            Conv2d-5              [-1, 2, 3, 3]           1,152
         AvgPool2d-6              [-1, 2, 1, 1]               0
Total params: 296,192
Trainable params: 296,192
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.10
Forward/backward pass size (MB): 0.05
Params size (MB): 1.13
Estimated Total Size (MB): 1.27
----------------------------------------------------------------


In [26]:
from tqdm import tqdm
from functools import partial
import torch.nn.functional as F
tqdm = partial(tqdm, leave=True, position=0)
model = Model()
model.to('cuda')
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
# criterion = nn.CrossEntropyLoss().to('cuda')





def train(model, train_loader, optimizer):
    model.train()
    train_loss = 0
    correct = 0
    conf_mat = np.zeros((2,2))
    all_target_labels = []
    all_pred_labels = []



    for _, data in tqdm(enumerate(train_loader), total=len(train_loader)):

        # update the counts for 0 and 1 in the current batch
        target = data['target']
        all_target_labels.extend(target.numpy())
        data, target = data['data'].to('cuda'),target.to('cuda')

        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        train_loss += loss.detach()
        loss.backward()
        optimizer.step()

        pred = output.argmax(dim=1, keepdim=True)
        all_pred_labels.extend(pred.cpu().numpy())

        
            
        # overall accuracy
        correct += pred.eq(target.view_as(pred)).sum().item()
    
    # return train_loss, correct
    for (i,j) in zip(all_target_labels, all_pred_labels):
        conf_mat[i,j] += 1
    print(
            f"Training loss = {train_loss * 1.0 / len(train_loader.dataset)},Training Accuracy : {100.0 * correct / len(train_loader.dataset)}, Class 1 accuracy: { conf_mat[1,1]/(conf_mat[1,1] + conf_mat[1,0])}, Class 0 accuracy : {conf_mat[0,0]/(conf_mat[0,0] + conf_mat[0,1])}", 
        )

def evaluate(model, valid_loader):
    model.eval()
    valid_loss = 0
    correct = 0
    conf_mat = np.zeros((2,2))
    all_target_labels = []
    all_pred_labels = []
    with torch.no_grad():
        for _, data in tqdm(enumerate(valid_loader), total=len(valid_loader)):
            target = data['target']
            all_target_labels.extend(target.numpy())
            data, target = data['data'].to('cuda'), target.to('cuda')
            output = model(data)
            valid_loss +=F.nll_loss(output, target, reduction='sum').item()

            pred = output.argmax(dim=1, keepdim=True)
            all_pred_labels.extend(pred.cpu().numpy())
            correct += pred.eq(target.view_as(pred)).sum().item()
    
    for (i,j) in zip(all_target_labels, all_pred_labels):
        conf_mat[i,j] += 1
    print(
            f"Valid loss = {valid_loss * 1.0 / len(valid_loader.dataset)}, Valid Accuracy : {100.0 * correct / len(valid_loader.dataset)} Class 1 accuracy: { conf_mat[1,1]/(conf_mat[1,1] + conf_mat[1,0])}, Class 0 accuracy : {conf_mat[0,0]/(conf_mat[0,0] + conf_mat[0,1])}"
        )

for i in range(10):
    print(f'EPOCH {i+1}')
    train(model, train_loader, optimizer)
    evaluate(model, valid_loader)


EPOCH 1


100%|██████████| 63/63 [00:00<00:00, 213.76it/s]


Training loss = 0.019314173609018326,Training Accuracy : 86.6, Class 1 accuracy: 0.836, Class 0 accuracy : 0.896


100%|██████████| 50/50 [00:00<00:00, 271.74it/s]


Valid loss = 0.23307672169059515, Valid Accuracy : 89.625 Class 1 accuracy: 0.875, Class 0 accuracy : 0.9175
EPOCH 2


...

Valid loss = 0.4036073768977076, Valid Accuracy : 89.25 Class 1 accuracy: 0.8725, Class 0 accuracy : 0.9125
EPOCH 10


100%|██████████| 63/63 [00:00<00:00, 214.63it/s]


Training loss = 0.0006236975896172225,Training Accuracy : 99.8, Class 1 accuracy: 0.996, Class 0 accuracy : 1.0


100%|██████████| 50/50 [00:00<00:00, 267.27it/s]

Valid loss = 0.4250976419710787, Valid Accuracy : 89.25 Class 1 accuracy: 0.9, Class 0 accuracy : 0.885
